In [1]:
import pandas as pd
import glob
import os

def read_csv_flexible(path: str) -> pd.DataFrame:
    """
    Читает CSV в нескольких кодировках (utf-8-sig, utf-8, latin1, cp1251, cp1252),
    явно sep="," чтобы правильно разбить колонки и отбросить BOM.
    """
    for enc in ("utf-8-sig", "utf-8", "latin1", "cp1251", "cp1252"):
        try:
            return pd.read_csv(path, sep=",", encoding=enc)
        except (UnicodeDecodeError, pd.errors.ParserError):
            continue
    raise UnicodeDecodeError(f"Не удалось прочитать файл {path} ни одной из кодировок")

def sample_per_file(df: pd.DataFrame,
                    total_rows: int = 100000,
                    label_col: str = "Label",
                    benign_value: str = "BENIGN") -> pd.DataFrame:
    """
    Берёт случайно до total_rows строк из df: 70% BENIGN, 30% — остальные метки.
    Если строк одной из групп меньше, уменьшает выборку до доступного минимума.
    """
    count_benign = int(total_rows * 0.7)
    count_other = int(total_rows * 0.3)
    
    df_b = df[df[label_col] == benign_value]
    df_o = df[df[label_col] != benign_value]
    
    count_benign = min(len(df_b), count_benign)
    count_other = min(len(df_o), count_other)

    if count_benign == 0 and count_other == 0:
        return pd.DataFrame()  # пустая выборка

    if count_benign < int(total_rows * 0.7) or count_other < int(total_rows * 0.3):
        print(f"⚠️ Всего в файле {len(df)} строк: {len(df_b)} BENIGN, {len(df_o)} атак; "
              f"взято {count_benign} BENIGN и {count_other} атак.")

    s_b = df_b.sample(n=count_benign, random_state=43)
    s_o = df_o.sample(n=count_other, random_state=43)
    return pd.concat([s_b, s_o], ignore_index=True)


def process_folder(input_folder: str, output_folder: str):
    os.makedirs(output_folder, exist_ok=True)
    paths = sorted(glob.glob(os.path.join(input_folder, "*.csv")))
    if not paths:
        print("В папке нет CSV-файлов.")
        return

    # Первый проход: сохраняем *_sampled1.csv с Label
    for path in paths:
        fname = os.path.basename(path)
        print(f"\nОбрабатываю {fname} …")
        df = read_csv_flexible(path)
        df.columns = df.columns.str.strip()

        # Сортируем исходный DF по Timestamp, если он есть
        if "Timestamp" in df.columns:
            df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
            df = df.sort_values("Timestamp").reset_index(drop=True)
        else:
            print("  ⚠ Нет столбца 'Timestamp' — сортировка исходного DF пропущена.")

        if "Label" not in df.columns:
            print("  ⚠ Нет столбца 'Label' — пропускаю файл.")
            continue

        sampled1 = sample_per_file(df)
        if sampled1.empty:
            print("  ⚠ Выборка пуста (нет BENIGN или нет атак) — пропускаю файл.")
            continue

        # Сортируем выборку по Timestamp, если он есть
        if "Timestamp" in sampled1.columns:
            sampled1 = sampled1.sort_values("Timestamp").reset_index(drop=True)

        name1 = os.path.splitext(fname)[0] + "_sampled_v3.csv"
        sampled1.to_csv(os.path.join(output_folder, name1), index=False)

        # Выводим диапазон времени, только если есть Timestamp
        if "Timestamp" in sampled1.columns:
            tmin = sampled1["Timestamp"].min()
            tmax = sampled1["Timestamp"].max()
            print(f"  ✅ Сохранён {name1}: {len(sampled1)} строк (время от {tmin} до {tmax})")
        else:
            print(f"  ✅ Сохранён {name1}: {len(sampled1)} строк")

    # Второй проход: опционально — без Label
    ans = input("\nСоздать второй вариант выборки и убрать столбец Label? (да/нет): ").strip().lower()
    if ans in ("да", "yes"):
        for path in paths:
            fname = os.path.basename(path)
            print(f"\n[2-й вариант] Обрабатываю {fname} …")
            df = read_csv_flexible(path)
            df.columns = df.columns.str.strip()

            if "Timestamp" in df.columns:
                df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
                df = df.sort_values("Timestamp").reset_index(drop=True)

            if "Label" not in df.columns:
                print("  ⚠ Нет столбца 'Label' — пропускаю файл.")
                continue

            sampled2 = sample_per_file(df)
            if sampled2.empty:
                print("  ⚠ Пустая выборка — пропускаю файл.")
                continue

            if "Timestamp" in sampled2.columns:
                sampled2 = sampled2.sort_values("Timestamp").reset_index(drop=True)

            sampled2 = sampled2.drop(columns=["Label"])
            name2 = os.path.splitext(fname)[0] + "_sampled_v4.csv"
            sampled2.to_csv(os.path.join(output_folder, name2), index=False)

            if "Timestamp" in sampled2.columns:
                tmin = sampled2["Timestamp"].min()
                tmax = sampled2["Timestamp"].max()
                print(f"  ✅ Сохранён {name2}: {len(sampled2)} строк (время от {tmin} до {tmax})")
            else:
                print(f"  ✅ Сохранён {name2}: {len(sampled2)} строк")

if __name__ == "__main__":
    inp = input("Путь к папке с исходными CSV: ").strip()
    outp = input("Путь, куда сохранять выборки: ").strip()
    if not os.path.isdir(inp):
        print("❌ Входная папка не найдена.")
    else:
        process_folder(inp, outp)


Путь к папке с исходными CSV:  C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\GeneratedLabelledFlows\TrafficLabelling
Путь, куда сохранять выборки:  C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\Code\diplom-project\diplom\test\result\Date



Обрабатываю Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv …
  ✅ Сохранён Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_sampled_v3.csv: 100000 строк (время от 2017-07-07 03:30:00 до 2017-07-07 05:02:00)

Обрабатываю Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv …
  ✅ Сохранён Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_sampled_v3.csv: 100000 строк (время от 2017-07-07 01:00:00 до 2017-07-07 03:29:00)

Обрабатываю Friday-WorkingHours-Morning.pcap_ISCX.csv …
⚠️ Всего в файле 191033 строк: 189067 BENIGN, 1966 атак; взято 70000 BENIGN и 1966 атак.
  ✅ Сохранён Friday-WorkingHours-Morning.pcap_ISCX_sampled_v3.csv: 71966 строк (время от 2017-07-07 08:59:00 до 2017-07-07 12:59:00)

Обрабатываю Monday-WorkingHours.pcap_ISCX.csv …
⚠️ Всего в файле 529918 строк: 529918 BENIGN, 0 атак; взято 70000 BENIGN и 0 атак.
  ✅ Сохранён Monday-WorkingHours.pcap_ISCX_sampled_v3.csv: 70000 строк (время от 2017-03-07 01:00:03 до 2017-03-07 12:59:54)

Обрабатываю Thursday-WorkingHours-Afternoon-I


Создать второй вариант выборки и убрать столбец Label? (да/нет):  да



[2-й вариант] Обрабатываю Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv …
  ✅ Сохранён Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_sampled_v4.csv: 100000 строк (время от 2017-07-07 03:30:00 до 2017-07-07 05:02:00)

[2-й вариант] Обрабатываю Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv …
  ✅ Сохранён Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_sampled_v4.csv: 100000 строк (время от 2017-07-07 01:00:00 до 2017-07-07 03:29:00)

[2-й вариант] Обрабатываю Friday-WorkingHours-Morning.pcap_ISCX.csv …
⚠️ Всего в файле 191033 строк: 189067 BENIGN, 1966 атак; взято 70000 BENIGN и 1966 атак.
  ✅ Сохранён Friday-WorkingHours-Morning.pcap_ISCX_sampled_v4.csv: 71966 строк (время от 2017-07-07 08:59:00 до 2017-07-07 12:59:00)

[2-й вариант] Обрабатываю Monday-WorkingHours.pcap_ISCX.csv …
⚠️ Всего в файле 529918 строк: 529918 BENIGN, 0 атак; взято 70000 BENIGN и 0 атак.
  ✅ Сохранён Monday-WorkingHours.pcap_ISCX_sampled_v4.csv: 70000 строк (время от 2017-03-07 01:00:03 до 2017-03-07 